# Data Ingestion

Exploring the arxiv API to collect our context dataset.

We use the same poetry env from the modules/data_ingestion folder. To activate it here, navigate to that directory and run 

```bash
poetry shell
```

inside WSL. Then run

```bash
jupyter notebook
```

and you will get a local host link. Next, in a notebook (in any directory) click on the `select_kernel` button at the top right, click `Select Another Kernel`, and then `Existing Jupyter Server`. Copy and paste the local host link in there and press enter.

In [36]:
pwd

'/mnt/c/Users/RaviB/GitHub/vegan-ai-nutritionist'

In [ ]:
import os

# Switch to the directory containing the pyproject.toml file
os.chdir("../modules/data_ingestion")

# Install libraries using poetry, uncomment and change library names as needed
#!poetry add arxiv

#switch back to the original directory
os.chdir("../../notebooks")

In [16]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching a keyword.
search = arxiv.Search(
  query = "quantum",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

In [17]:
results = client.results(search)

In [18]:
results

In [19]:
for result in results:
    print(result)

http://arxiv.org/abs/2409.02921v1
http://arxiv.org/abs/2409.02916v1
http://arxiv.org/abs/2409.02911v1
http://arxiv.org/abs/2409.02909v1
http://arxiv.org/abs/2409.02900v1
http://arxiv.org/abs/2409.02899v1
http://arxiv.org/abs/2409.02898v1
http://arxiv.org/abs/2409.02892v1
http://arxiv.org/abs/2409.02887v1
http://arxiv.org/abs/2409.02881v1


In [20]:
next(results)

StopIteration: 

In [3]:
papers = []

for result in results:
    title = result.title
    summary = result.summary
    url = result.pdf_url
    data_published = paper.published
    categories = paper.categories
    
    paper = {
        "title": title,
        "summary": summary,
        "url": url,
        "date_published": data_published,
        "categories": categories
    }
    
    papers.append(paper)

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [10]:
pap = next(results)

In [ ]:
pap

arxiv.Result(entry_id='http://arxiv.org/abs/2408.11877v1', updated=datetime.datetime(2024, 8, 20, 14, 44, 56, tzinfo=datetime.timezone.utc), published=datetime.datetime(2024, 8, 20, 14, 44, 56, tzinfo=datetime.timezone.utc), title='Towards an Accessible, Noninvasive Micronutrient Status Assessment Method: A Comprehensive Review of Existing Techniques', authors=[arxiv.Result.Author('Andrew Balch'), arxiv.Result.Author('Maria A. Cardei'), arxiv.Result.Author('Sibylle Kranz'), arxiv.Result.Author('Afsaneh Doryab')], summary='Nutrients are critical to the functioning of the human body and their\nimbalance can result in detrimental health concerns. The majority of\nnutritional literature focuses on macronutrients, often ignoring the more\ncritical nuances of micronutrient balance, which require more precise\nregulation. Currently, micronutrient status is routinely assessed via complex\nmethods that are arduous for both the patient and the clinician. To address the\nglobal burden of micronut

In [ ]:
next(pap)

In [60]:
pap.pdf_url

'http://arxiv.org/pdf/2408.11877v1'

In [41]:
papers[0]

IndexError: list index out of range

In [32]:
import requests
from PyPDF2 import PdfReader
from io import BytesIO

def get_pdf_text(url):
    response = requests.get(url)
    pdf_bytes = BytesIO(response.content)
    pdf = PdfReader(pdf_bytes)
    
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [61]:
print(get_pdf_text('https://arxiv.org/pdf/2408.11877v1'))

TOWARDS AN ACCESSIBLE , NONINVASIVE MICRONUTRIENT
STATUS ASSESSMENT METHOD : A C OMPREHENSIVE REVIEW OF
EXISTING TECHNIQUES
Andrew Balch
University of Virginia
Charlottesville, V A 22903
xxv2zh@virginia.eduMaria A. Cardei
University of Virginia
Charlottesville, V A 22903
cbr8ru@virginia.eduSibylle Kranz
University of Virginia
Charlottesville, V A 22903
sk4gd@virginia.eduAfsaneh Doryab
University of Virginia
Charlottesville, V A 22903
ad4ks@virginia.edu
August 23, 2024
ABSTRACT
Nutrients are critical to the functioning of the human body and their imbalance can result in detrimental
health concerns. The majority of nutritional literature focuses on macronutrients, often ignoring the
more critical nuances of micronutrient balance, which require more precise regulation. Currently,
micronutrient status is routinely assessed via complex methods that are arduous for both the patient and
the clinician. To address the global burden of micronutrient malnutrition, innovations in assessment
must b